<a href="https://colab.research.google.com/github/SakshiGehani/J015_NLPassignments/blob/HomeWork/HW9_TopicModelling_J015_13_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topic Modelling using nmf and lda

In [0]:
import os

# Install Kaggle library
!pip install -q kaggle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/My Drive/.kaggle/'

In [4]:
!kaggle datasets download -d hj5992/nlp-topic-modelling

 98% 112M/115M [00:01<00:00, 61.4MB/s]
100% 115M/115M [00:01<00:00, 71.9MB/s]


In [5]:
!unzip nlp-topic-modelling.zip

Archive:  nlp-topic-modelling.zip
  inflating: Reviews.csv             


In [6]:
import pandas as pd

df = pd.read_csv("Reviews.csv")
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [0]:
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from gensim.models import ldamodel
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import pickle;

In [0]:
df = df.iloc[:2000,]

In [9]:
df.shape

(2000, 10)

In [10]:
text = df.Summary
text

0                Good Quality Dog Food
1                    Not as Advertised
2                "Delight" says it all
3                       Cough Medicine
4                          Great taffy
                     ...              
1995    Great Waffles for us non-cooks
1996    Excellent Fluffy Pancakes.....
1997                    Good Basic Mix
1998       Good pancakes, lots of work
1999            waffles, schmaffles !!
Name: Summary, Length: 2000, dtype: object

In [11]:
# importing required modules
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def lemma(pos):
  lem = WordNetLemmatizer()
  lemma = []
  for word in pos:
    for w in word:
      pos_value = ""
      if (w[1].startswith('J')):
        pos_value = wordnet.ADJ
      elif (w[1].startswith('V')):
        pos_value = wordnet.VERB
      elif (w[1].startswith('N')):
        pos_value = wordnet.NOUN
      elif (w[1].startswith('R')):
        pos_value = wordnet.ADV 
      else:
        continue
      lemma.append(lem.lemmatize(w[0],pos_value))
  return (lemma)

In [0]:
## Preprocessing data

stop_words = set(stopwords.words('english'))      # for unique entries -> set()
#stop_words.add("nan")
words = set(nltk.corpus.words.words())
def preprocess(sen):
  sen = sen.lower()
  no_punc = sen.translate(sen.maketrans('', '', '!"$%&\'()#@*+,-./:;<=>?[\\]^_`{|}~'))      ## removing all punctuation marks
  filtered = no_punc.translate(no_punc.maketrans('', '', '#@0123456789'))           ## removing digits
  tokens = word_tokenize(filtered)                        ## creating a list of tokens
  no_stop_words = [w for w in tokens if not w in stop_words]     # eliminate stop words;  returns list of lists
  
  pos = []
  pos.append(nltk.pos_tag(no_stop_words))
  lemmatized = lemma(pos)
  sent = " ".join(w for w in lemmatized)
  
  return sent

In [0]:
df["clean_text"] = text.apply(preprocess)
clean_text = df.clean_text

In [15]:
df.clean_text 

0          good quality dog food
1                     advertised
2                    delight say
3                 cough medicine
4                    great taffy
                  ...           
1995       great waffle noncooks
1996    excellent fluffy pancake
1997              good basic mix
1998       good pancake lot work
1999           waffle schmaffles
Name: clean_text, Length: 2000, dtype: object

In [16]:
clean_text

0          good quality dog food
1                     advertised
2                    delight say
3                 cough medicine
4                    great taffy
                  ...           
1995       great waffle noncooks
1996    excellent fluffy pancake
1997              good basic mix
1998       good pancake lot work
1999           waffle schmaffles
Name: clean_text, Length: 2000, dtype: object

In [0]:
dataset = [d.split() for d in df.clean_text]

In [0]:
# Initialize the number of Topics we need to cluster
num_topics = 10

# First, we obtain a id-2-word dictionary. For each headline, we will use the dictionary to obtain a mapping of the word id to their word counts.
id2word = gensim.corpora.Dictionary(dataset);
corpus = [id2word.doc2bow(txt) for txt in dataset];
lda = ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics);

In [0]:
#Generating LDA topics
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [20]:
get_lda_topics(lda, num_topics)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,good,tea,great,great,love,good,good,chip,good,tasty
1,love,best,chip,best,yum,delicious,pop,pop,chip,good
2,great,delicious,best,love,taste,best,chip,love,great,great
3,taste,ever,taste,favorite,yummy,chip,best,best,love,awesome
4,product,great,dog,popchips,chip,flavor,love,cat,well,chip
5,dog,food,snack,taste,best,ever,hard,excellent,taste,best
6,food,yummy,tea,product,tea,taste,taste,awful,delicious,excellent
7,chip,good,love,hot,product,salt,delicious,yum,product,quality
8,flavor,product,potato,chip,ever,coffee,tasty,snack,healthy,favorite
9,cute,chip,good,tasty,great,food,price,perfect,stuff,food


#***using NMF to get topics*** 

In [21]:
clean_text

0          good quality dog food
1                     advertised
2                    delight say
3                 cough medicine
4                    great taffy
                  ...           
1995       great waffle noncooks
1996    excellent fluffy pancake
1997              good basic mix
1998       good pancake lot work
1999           waffle schmaffles
Name: clean_text, Length: 2000, dtype: object

In [0]:
#To reduce the size of the matrix, to speed up computation, we will set the maximum feature size to 5000, which will take the top 5000 best features that can contribute to our model.
vectorizer = CountVectorizer(analyzer='word', max_features=5000);
X_counts = vectorizer.fit_transform(clean_text);
#Next, we set a TfIdf Transformer, and transform the counts with the model.
transformer = TfidfTransformer(smooth_idf=False);
X_tfidf = transformer.fit_transform(X_counts);

In [0]:
#And now we normalize the TfIdf values to unit length for each row.
X_tfidf_norm = normalize(X_tfidf, norm='l1', axis=1)

In [24]:
#And finally, obtain a NMF model, and fit it with the sentences.

#obtain a NMF model.
model = NMF(n_components=num_topics, init='nndsvd');
#fit the model
model.fit(X_tfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=10, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
#Generating NMF topics:

def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [26]:
get_nmf_topics(model, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,good,delicious,love,great,best,yum,yummy,chip,tasty,popchips
1,taste,absolutely,dog,product,ever,pop,tummy,pop,easy,variety
2,flavor,healthy,cat,taste,tea,healthy,treat,excellent,organic,review
3,stuff,tea,stuff,snack,stevia,say,small,kettle,crunchy,rule
4,product,easy,food,food,sauce,falafel,different,favorite,healthy,luuuv
5,pretty,awesome,product,deal,coffee,want,oooh,awesome,salty,weightwatchers
6,coffee,product,snack,price,ive,something,yummies,potato,fresh,flavor
7,food,hot,son,stuff,flavor,snack,snack,ever,fruit,favorite
8,healthy,pretty,mueslix,dog,simply,really,potato,salt,beware,hella
9,really,omg,family,value,food,sweetener,healthy,addictive,sooo,wonderfully
